# Linear Regression with scikit-learn

This notebook creates and measures a linear regression model using sklearn.

* Method: Ordinary Least Squares
* Dataset: Boston housing data

## Imports

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

## Load the Data

In [ ]:
# Load the Boston housing data
boston = load_boston()

In [ ]:
# Get more information about the dataset including the feature names and meanings
print(boston.DESCR)

In [ ]:
# As the dataset is a dict view the keys
boston.keys()

In [ ]:
# Convert the dict into a Pandas dataframe
boston_df = pd.DataFrame(boston.data)
boston_df.head()

In [ ]:
# Replace the columns with the column names
boston_df.columns = boston.feature_names
boston_df.head()

In [ ]:
# Add the target prices to the dataframe
boston_df['PRICE'] = boston.target
boston_df.head()

## Fit a Linear Regression Model

* Y = Target variable; Boston housing price
* X = Independent variables; all other features

In [ ]:
# Create our feature dataframe by dropping the price column
X = boston_df.drop('PRICE', axis=1)
X.head()

In [ ]:
# Create a linear regression model
lm = LinearRegression()
lm

In [ ]:
# Split the dataset into training and testing datasets
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, boston_df.PRICE, test_size=0.33, random_state=5)

In [ ]:
# Fit (train) the model with the training data
lm.fit(X_train, Y_train)

**Intercept Coefficient**: represents the mean change in the response variable for one unit of change in the predictor variable while holding everything else constant. It isolates the role of one variable from all others.

In [ ]:
# Print the intercept coefficient
print('Estimated intercept coefficient: {}'.format(lm.intercept_))

In [ ]:
# Number of coefficients: 
print('Number of coefficients: {}'.format(len(lm.coef_)))

**Note**: the correlation coefficients (below) give an idea of the strength of the relationship between two variables.

In [ ]:
# Create a dataframe with the features and coefficients
fc_df = pd.DataFrame(list(zip(X.columns, lm.coef_)), columns=['features', 'coefficients'])
fc_df

**Interpretation**: it appears that there is a high positive correlation between RM and prices.

In [ ]:
# Create a plot for RM and housing prices
fig = plt.figure(figsize=(20,10))
plt.scatter(boston_df.RM, boston_df.PRICE)
plt.xlabel("Average number of rooms per dwelling (RM)")
plt.ylabel("Housing Price")
plt.title("Relationship between RM and Price")
plt.show()

## Predict a Price

In [ ]:
# Use the test data to create predictions and show the first 5
y_pred = lm.predict(X_test)
y_pred[0:5]

In [ ]:
# Create a plot to compare actual prices (Y_test) and the predicted prices (pred_test)
fig = plt.figure(figsize=(20,10))
plt.scatter(Y_test, y_pred)
plt.xlabel("Actual Prices: $Y_i$")
plt.ylabel("Predicted Prices: $\hat{Y}_i$")
plt.title("Actual vs. Predicted Prices: $Y_i$ vs. $\hat{Y}_i$")
plt.show()

**Observation**: there is error in the predictions as the housing price increases

## Model Evaluation

### Mean Squared Error

* A measure of the average magnitude of the errors without consideration for their direction; measures accuracy for continuous variables.
* Always non-negative
* Values closer to zero (0) are better

In [ ]:
# Get the Mean Squared Error (MSE) for all predictions
mse = mean_squared_error(Y_train, lm.predict(X_train))
print("MSE Training Data: {}".format(mse))

In [ ]:
# Get the MSE for the test data
print("MSE Test Data: {}".format(mean_squared_error(Y_test, lm.predict(X_test))))

In [ ]:
# Get the MSE for a single feature for comparison
lm2 = LinearRegression()
lm2.fit(X[['PTRATIO']], boston_df.PRICE)

mse2 = mean_squared_error(boston_df.PRICE, lm2.predict(X[['PTRATIO']]))
print("MSE All Data Single Feature: {}".format(mse2))

**Observation**: because the MSE increased for a single feature, this indicates that a single feature is not a good predictor of housing prices.

### Variance (R^2)

* Explains how much of the variability of a factor can be caused or explained by its relationship to another factor; how well the model is predicting.
* A score of 1 means a perfect prediction
* A score of 0 means the model always predicts the expected value of y, disregarding the input features

In [ ]:
print("Variance Score: %.2f" % r2_score(Y_test, y_pred))

### Residual Plot

**Residuals**: the difference between the predictions and the actuals.


**Interpretation**: If the model is working well then the data should be randomly scattered around line zero. If there is structure in the data, that means the model is not capturing something, perhaps interaction between two variables or it's time dependent. Check the parameters of your model.

In [ ]:
# Create a residual plot
fig = plt.figure(figsize=(20,10))
plt.scatter(lm.predict(X_train), lm.predict(X_train) - Y_train, c='b', s=40, alpha=0.5)
plt.scatter(lm.predict(X_test), lm.predict(X_test) - Y_test, c='g', s=40)
plt.hlines(y=0, xmin=0, xmax=50)
plt.ylabel("Residuals")
plt.title("Residual Plot Using Training (Blue) and Test (Green) Data")
plt.show()